# Trains and  tests on parts of  PATIENT1 data for Del_st, Del_end and No_sv, ground  truth nanosv_manta

## Load dependences and setting output configuration

In [234]:
import numpy as np
from keras.utils.np_utils import to_categorical
import keras
import gzip
%pylab inline
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/hpc/cog_bioinf/ridder/users/lsantuari/Software/miniconda2/envs/mcfly_new/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['f', 'colors']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


## Load data from npy files

In [235]:
import pickle
# dico = pickle.load(open("/hpc/cog_bioinf/ridder/users/lsantuari/Processed/Test/060818/TestData_060818/NA12878/MultiLabelData/labels.pickle", "rb"))
# labels = dataset = dico["Mills2011_nanosv"]

dico = pickle.load(open("/hpc/cog_bioinf/ridder/users/lsantuari/Processed/Test/060818/TestData_060818/PATIENT1/MultiLabelData/labels.pickle", "rb"))
print(dico.keys())
labels = dataset = dico["nanosv_manta"]

window_size = 200

dict_keys(['nanosv', 'nanosv_manta', 'id'])


In [236]:
dico = []

In [286]:
#datapath = '/hpc/cog_bioinf/ridder/users/lsantuari/Processed/Test/060818/TestData_060818/NA12878' 
datapath = '/hpc/cog_bioinf/ridder/users/lsantuari/Processed/Test/060818/TestData_060818/PATIENT1/'

germline_1_datapath = datapath + '/ChannelData/'
ger_1_labels_datapath = datapath + '/LabelData/'

G1_data_SV = np.empty([0, 27, window_size])
G1_labels_SV =[]
G1_data_noSV = np.empty([0, 27, window_size])
G1_labels_noSV =[]

for i in range(1,23,1):
        print('Loading Chr'+str(i))
        data_file = germline_1_datapath + str(i) +'_channel_maker_real_germline.npy.gz'
        with gzip.GzipFile(data_file, "rb") as f:
            G1_data_new = np.load(f)
        f.close()
    
        #G1_labels_new = np.load(ger_1_labels_datapath  +  str(i) +'_label_ci_full_overlap.npy/' +str(i) +'_label_ci_full_overlap.npy') 
        G1_labels_new = np.array(labels[str(i)])
        # print(G1_labels_new[0])
        G1_data_new = G1_data_new[np.where(G1_labels_new != 'UK')]
        G1_labels_new = G1_labels_new[np.where(G1_labels_new != 'UK')]
        G1_data_SV_new  = G1_data_new[np.where(G1_labels_new != 'noSV')]
        G1_labels_SV_new  = G1_labels_new[np.where(G1_labels_new != 'noSV')]
        n_windows_sv = G1_data_SV_new.shape[0]
        G1_data_noSV_new  = G1_data_new[np.where(G1_labels_new == 'noSV')]
        G1_labels_noSV_new  = G1_labels_new[np.where(G1_labels_new == 'noSV')]
        G1_data_noSV_new =  G1_data_noSV_new[0:n_windows_sv]
        G1_labels_noSV_new =  G1_labels_noSV_new[0:n_windows_sv]
        print(str(i) + ':' + str(G1_data_SV_new.shape))
        # print(G1_labels_SV_new[0])
        G1_data_SV = np.concatenate((G1_data_SV, G1_data_SV_new))
        G1_labels_SV = np.concatenate((G1_labels_SV, G1_labels_SV_new))
        G1_data_noSV = np.concatenate((G1_data_noSV, G1_data_noSV_new))
        G1_labels_noSV = np.concatenate((G1_labels_noSV, G1_labels_noSV_new))

Loading Chr1
1:(255, 27, 200)
Loading Chr2
2:(298, 27, 200)
Loading Chr3
3:(247, 27, 200)
Loading Chr4
4:(305, 27, 200)
Loading Chr5
5:(233, 27, 200)
Loading Chr6
6:(248, 27, 200)
Loading Chr7
7:(239, 27, 200)
Loading Chr8
8:(210, 27, 200)
Loading Chr9
9:(146, 27, 200)
Loading Chr10
10:(164, 27, 200)
Loading Chr11
11:(176, 27, 200)
Loading Chr12
12:(176, 27, 200)
Loading Chr13
13:(133, 27, 200)
Loading Chr14
14:(105, 27, 200)
Loading Chr15
15:(84, 27, 200)
Loading Chr16
16:(91, 27, 200)
Loading Chr17
17:(91, 27, 200)
Loading Chr18
18:(105, 27, 200)
Loading Chr19
19:(71, 27, 200)
Loading Chr20
20:(45, 27, 200)
Loading Chr21
21:(61, 27, 200)
Loading Chr22
22:(55, 27, 200)


### Find  the indexes that split the categories

In [287]:
def find_splitting_indices(labels):
    for i in range (0, labels.shape[0] - 1):
        if (labels[i] != labels[i+1]):
            print(i , ': ', labels[i] , labels[i+1] )        

In [288]:
#find_splitting_indices(G1_labels)

In [289]:
#G1_labels_SV = ['DEL'] * G1_labels_SV.shape[0]

In [290]:
sv=G1_data_SV.shape[0]

In [291]:
sv

3538

In [292]:
int(sv/2)

1769

In [293]:
G1_data_noSV  = G1_data_noSV[0:int(sv/2)] 

In [294]:
G1_labels_noSV  = G1_labels_noSV[0:int(sv/2)] 

In [295]:
SV_train = 1600
noSV_train = 800
SV_val = 350
noSV_val = 175

In [296]:
X_train = np.concatenate((G1_data_SV[0:SV_train], G1_data_noSV[0:noSV_train ]))
X_val = np.concatenate((G1_data_SV[SV_train:SV_train+SV_val], G1_data_noSV[noSV_train:noSV_train +noSV_val ]))
X_test = np.concatenate((G1_data_SV[SV_train+SV_val:], G1_data_noSV[noSV_train +noSV_val:]))

In [297]:
y_train = np.concatenate((G1_labels_SV[0:SV_train], G1_labels_noSV[0:noSV_train ]))
y_val = np.concatenate((G1_labels_SV[SV_train:SV_train+SV_val], G1_labels_noSV[noSV_train:noSV_train +noSV_val ]))
y_test = np.concatenate((G1_labels_SV[SV_train+SV_val:], G1_labels_noSV[noSV_train +noSV_val:]))

### Number of training, validation and test windows

In [298]:
X_train.shape

(2400, 27, 200)

In [299]:
X_val.shape

(525, 27, 200)

In [300]:
X_test.shape

(2382, 27, 200)

In [301]:
y_test.shape

(2382,)

In [302]:
number_channels = X_train.shape[1]
number_channels

27

## Normalize data using the reads coverage as parameter and remove uninteresting channels for deletions

In [303]:
coverage = np.median(X_train[:, 0, :]) #the median of the coverage in the normal cells
coverage


33.0

In [304]:
def normalize(X):       
    for i in range (0, len(X)):                              
        for j in range(0,number_channels):                                                                                            
            X[i][j] = np.divide(X[i][j], coverage)*100 
# normalize(X_train)
# normalize(X_val)
# normalize(X_test)

In [305]:
def eliminate_noise_chanels(X):
    return np.delete(X, [4-1,5-1,6-1,7-1,8-1,9-1,10-1,12-1,13-1,15-1,16-1, 17-1,18-1,19-1,23-1,24-1,26-1,27-1], axis=1)
    

In [306]:
def eliminate_distance_chanels(X):
    return np.delete(X, [3,4,7,8], axis=1)

In [307]:
X_train = eliminate_noise_chanels(X_train)
X_val = eliminate_noise_chanels(X_val)
X_test = eliminate_noise_chanels(X_test)

In [308]:
X_train = eliminate_distance_chanels(X_train)
X_val = eliminate_distance_chanels(X_val)
X_test = eliminate_distance_chanels(X_test)

In [309]:
X_train.shape

(2400, 5, 200)

In [310]:
number_channels = X_train.shape[1]
number_channels

5

## Differentiate the time series for a zero mean

In [311]:
def differentiate(X):
    for i in range(0, X.shape[0]):        
        for j in range(0, X.shape[1]):
            for k in range(0, X.shape[2]-1):
                X[i][j][k] = X[i][j][k+1] - X[i][j][k]
    return X[:,:,:-1]        

In [312]:
# X_train = differentiate(X_train)

In [313]:
# X_val = differentiate(X_val)
# X_test = differentiate(X_test)

## Plots several  windows

In [314]:
n_windows = 100
label = ["None"] * number_channels

In [315]:
label[0] = "germline:coverage"
label[1] = "germline:#left clipped reads"
label[2] = "germline:#right clipped reads"
label[3] = "germline:#split reads right split"
label[4] = "germline:#split reads left split"

In [316]:
from numpy import *
import math
import matplotlib.pyplot as plt

colors =['b','g', 'm', 'gray', 'r','c', 'darkgreen' ,'y', 'k', 'orange', 'gray']

def plot_channels(start_window, n_windows):
    for i in range (start_window, start_window + n_windows, 1):
        if y_test[i] == labels[predicted[i]]:
            continue
        #print(y_train[i], 'id:', i)
        print('id:', i)
        print('Nanopore:', y_test[i], '. DeepSV:', labels[predicted[i]])
        for j in range (0, number_channels):   
            shift = 0
            start =0
            if j in [0,11]:
                shift = -100
            if j > 8:
                start = -60
            Z = [start + shift + 300 + x - 5*j*4 for x in X_test[i][j]]  
            plt.ylim([150,450])
            if np.max(Z) > 1000:
                plt.ylim([-200,1000])
                Z = [start + shift + 300 + x - 5*j*4 - 400 for x in X_test[i][j]]  
            if j>8:
                plt.plot(Z,  label = label[j], linestyle=':', linewidth=1.5, color = colors[j % 9])
            else:
                plt.plot(Z,  label = label[j], color = colors[j])
        
        plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., prop={'size': 10})    
        plt.show()

In [317]:
start_window = 0
n_windows = 573
# plot_channels(start_window, n_windows)

In [318]:
n_windows = 50
start_window = 500
# plot_channels(start_window, n_windows)

   
## Transposes every window in X, to comply to McFly format

In [319]:
def transposeDataset(X):
    image = []
    for i in range (0, len(X -1)):
        tr = X[i].transpose()
        image.append(tr)
    return np.array(image)

image1 = transposeDataset(X_train)
image2 = transposeDataset(X_val)
image3 = transposeDataset(X_test)
X_train = image1
X_val = image2
X_test = image3

n_channels = len(X_train[0][0])
#print(len(X_test[0][0]))
print(X_test.shape)

(2382, 200, 5)


## Prepare the labels in McFly format

In [320]:
mapclasses = {'DEL_start': 1, 'DEL_end': 0,  'noSV': 2}
print(mapclasses)

{'DEL_start': 1, 'DEL_end': 0, 'noSV': 2}


In [321]:
y_train = np.array([mapclasses[c] for c in y_train], dtype='int')
y_val = np.array([mapclasses[c] for c in y_val], dtype='int')
y_test = np.array([mapclasses[c] for c in y_test], dtype='int')
y_train_binary = to_categorical(y_train)
y_val_binary = to_categorical(y_val)
y_test_binary = to_categorical(y_test)

In [322]:
y_test_binary

array([[1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [323]:
y_val_binary

array([[1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [324]:
X_train.shape

(2400, 200, 5)

##  Generate and train neural networks 

In [325]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Convolution1D, Flatten, MaxPooling1D
from keras.optimizers import Adam
import sys
import os
import numpy as np
sys.path.insert(0, os.path.abspath('..'))
from mcfly import modelgen, find_architecture

In [326]:
np.random.seed = 321
num_classes = y_train_binary.shape[1]

%%time
models = modelgen.generate_models(X_train.shape,
                                  num_classes,
                                  number_of_models = 1,
                                  model_type = 'CNN',
                                  cnn_min_layers=1,
                                  cnn_max_layers=1,
                                  cnn_min_filters = 3, 
                                  cnn_max_filters = 3, 
                                  cnn_min_fc_nodes=2, 
                                  cnn_max_fc_nodes=2,
                                  low_lr=1.73, high_lr=1.73,
                                  kernel_size = 11)

In [327]:
%%time
models = modelgen.generate_models(X_train.shape,
                                  num_classes,
                                  number_of_models = 1,
                                  model_type = 'CNN',
                                  cnn_min_layers=2,
                                  cnn_max_layers=2,
                                  cnn_min_filters = 4, 
                                  cnn_max_filters = 4, 
                                  cnn_min_fc_nodes=6, 
                                  cnn_max_fc_nodes=6,
                                  low_lr=2, high_lr=2)
                                  #kernel_size = 7)

CPU times: user 602 ms, sys: 6.92 ms, total: 609 ms
Wall time: 608 ms


In [328]:
%%time
i=0
for model, params, model_types in models:
    print('model ' + str(i))
    i=i+1
    print(params)
    model.summary()

model 0
{'learning_rate': 0.01, 'regularization_rate': 0.010014443502238768, 'filters': array([4, 4]), 'fc_hidden_nodes': 6}
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_25 (Batc (None, 200, 5)            20        
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 200, 4)            64        
_________________________________________________________________
batch_normalization_26 (Batc (None, 200, 4)            16        
_________________________________________________________________
activation_25 (Activation)   (None, 200, 4)            0         
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 200, 4)            52        
_________________________________________________________________
batch_normalization_27 (Batc (None, 200, 4)            16        
_________________

In [329]:
train_set_size = X_train.shape[0]
train_set_size

2400

In [330]:
%%time
histories, val_accuracies, val_losses = find_architecture.train_models_on_samples(X_train, y_train_binary,
                                                                                 X_val, y_val_binary,
                                                                                 models,nr_epochs=1, 
                                                                                 subset_size=train_set_size,
                                                                                 verbose=True)

Training model 0 CNN
Train on 2400 samples, validate on 525 samples
Epoch 1/1
2400/2400 [==============================] - 4s 2ms/step - loss: 0.6152 - acc: 0.8487 - val_loss: 0.7022 - val_acc: 0.7581
CPU times: user 6.38 s, sys: 315 ms, total: 6.69 s
Wall time: 5.36 s


## Save the best model

In [331]:
best_model_index = np.argmax(val_accuracies)
best_model, best_params, best_model_types = models[best_model_index]
print(best_model_index, best_model_types, best_params)

0 CNN {'learning_rate': 0.01, 'regularization_rate': 0.010014443502238768, 'filters': array([4, 4]), 'fc_hidden_nodes': 6}


In [332]:
from keras.models import load_model

model_path = os.path.join('/hpc/cog_bioinf/ridder/users/lsantuari/Processed/Test/060818/TestData_060818/NA12878/ModelData', 'Gtcg_13_9')

best_model.save(model_path)

## Train the best model on the full dataset

In [333]:
#We make a copy of the model, to start training from fresh
nr_epochs = 1
datasize = train_set_size # Change in `X_train.shape[0]` if training complete data set
history = best_model.fit(X_train[:datasize,:,:], y_train_binary[:datasize,:],
              epochs=nr_epochs, validation_data=(X_val, y_val_binary))

Train on 2400 samples, validate on 525 samples
Epoch 1/1
2400/2400 [==============================] - 1s 380us/step - loss: 0.3951 - acc: 0.9004 - val_loss: 1.3592 - val_acc: 0.3524


## Reload the trained model

In [334]:
import os
from keras.models import load_model
model_path = os.path.join('/hpc/cog_bioinf/ridder/users/lsantuari/Processed/Test/060818/TestData_060818/NA12878/ModelData',  'Gtcg_13_9')
model_reloaded = load_model(model_path)

## Inspect model predictions on test data

In [335]:

model = model_reloaded 
datasize = X_test.shape[0]
probs = model.predict_proba(X_test[:datasize,:,:],batch_size=1)

## Columns are predicted labels by DeepSV, rows are nanopore labels

In [336]:
mapclasses = {'DEL_end': 0, 'DEL_start': 1, 'noSV': 2}
dict=mapclasses
dict_sorted = sorted(dict.items(), key=lambda x: x[1])
dict_sorted

[('DEL_end', 0), ('DEL_start', 1), ('noSV', 2)]

In [337]:
labels = [i[0] for i in dict_sorted]

In [338]:
import pandas as pd
predicted = probs.argmax(axis=1)
y_index = y_test_binary.argmax(axis=1)
confusion_matrix = pd.crosstab(pd.Series(y_index), pd.Series(predicted))
confusion_matrix.index = [labels[i] for i in confusion_matrix.index]
confusion_matrix.columns = [labels[i] for i in confusion_matrix.columns]
confusion_matrix.reindex(columns=[l for l in labels], fill_value=0)
confusion_matrix

,DEL_end,DEL_start,noSV
DEL_end,550,2,253
DEL_start,10,377,396
noSV,7,7,780


## Test best current model on testset

### Overall accuracy on the test set:

In [339]:
%%time
score_test = model.evaluate(X_test, y_test_binary, verbose=False)
print('Test loss and accuracy of best model: ' + str(score_test))

Test loss and accuracy of best model: [0.7951599833446826, 0.716624685138539]
CPU times: user 1.22 s, sys: 55.4 ms, total: 1.28 s
Wall time: 1.02 s


In [340]:
df_conf = pd.DataFrame()

for i in np.linspace(1.0 / len(labels), 1, num=50, endpoint=False):

    predicted = np.argwhere(probs > i)[:, 1]
    y_index = np.argwhere(y_test_binary > i)[:, 1]

    # Rows: true, columns: predicted
    confusion_matrix = pd.crosstab(pd.Series(y_index), pd.Series(predicted))
    confusion_matrix.index = [labels[i] for i in confusion_matrix.index]
    confusion_matrix.columns = [labels[i] for i in confusion_matrix.columns]
    confusion_matrix.reindex(columns=[l for l in labels], fill_value=0)

    #print('Confusion matrix:')
    #print(confusion_matrix)
    
    for l in labels:
        if l in confusion_matrix.index:
            # print(confusion_matrix.loc[l,:])
            # print(confusion_matrix.loc[:,l])

            # label_correct = confusion_matrix.loc[l, l]
            label_precision = np.around(confusion_matrix.loc[l, l] / sum(confusion_matrix.loc[:, l]) * 100)
            label_recall = np.around(confusion_matrix.loc[l, l] / sum(confusion_matrix.loc[l, :]) * 100)
            label_F1 = 2 * (label_precision * label_recall) / (label_precision + label_recall)

            # print(f'Iter:{i} {l} -> Precision:{label_precision}%, Recall:{label_recall}%, F1:{label_F1}')

            df_intres = pd.DataFrame(
                {'iteration': [i], 'label': [l],
                 'precision': [label_precision], 'recall': [label_recall], 'F1': [label_F1]})
            df_conf = df_conf.append(df_intres)
            
print(df_conf)

    iteration      label  precision  recall          F1
0    0.333333    DEL_end       39.0    32.0   35.154930
0    0.333333  DEL_start       37.0    25.0   29.838710
0    0.333333       noSV       40.0    59.0   47.676768
0    0.346667    DEL_end       39.0    32.0   35.154930
0    0.346667  DEL_start       38.0    25.0   30.158730
0    0.346667       noSV       42.0    64.0   50.716981
0    0.360000    DEL_end       38.0    30.0   33.529412
0    0.360000  DEL_start       37.0    23.0   28.366667
0    0.360000       noSV       44.0    70.0   54.035088
0    0.373333    DEL_end       39.0    30.0   33.913043
0    0.373333  DEL_start       41.0    24.0   30.276923
0    0.373333       noSV       47.0    78.0   58.656000
0    0.386667    DEL_end       45.0    34.0   38.734177
0    0.386667  DEL_start       43.0    24.0   30.805970
0    0.386667       noSV       50.0    84.0   62.686567
0    0.400000    DEL_end       44.0    33.0   37.714286
0    0.400000  DEL_start       42.0    22.0   28

/hpc/cog_bioinf/ridder/users/lsantuari/Software/miniconda2/envs/mcfly_new/lib/python3.6/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in double_scalars


In [341]:
# df_conf[df_conf['label']=='DEL_start']

In [342]:
import matplotlib.pyplot as plt
#plt.plot(df_conf[df_conf['label']=='DEL_start']['precision'], df_conf[df_conf['label']=='DEL_start']['recall'])
#plt.ylabel('some numbers')
#plt.show()

#print(df_conf[df_conf['label']=='DEL_start']['precision'])

In [343]:
import pandas as pd
print(pd.__version__)

0.23.4
